In [1]:
import os
from pathlib import Path
from collections import namedtuple
from itertools import zip_longest

In [2]:
ROOM_DEPTH = 4

class Hall(namedtuple("Hall", ['slots'], defaults=[(None,)*11])):
    closed_slots = (2, 4, 6, 8)

    def enter(self, enter_index, one):
        for i in range(enter_index-1,-1,-1):        
            if i in Hall.closed_slots: 
                continue

            if self.slots[i] is not None:
                break

            moves = enter_index - i + 1
            yield moves, Hall(self.slots[:i] + (one, ) + self.slots[i+1:])

        for i in range(enter_index+1,len(self.slots)):
            if i in Hall.closed_slots: 
                continue

            if self.slots[i] is not None:
                break
            
            moves = i - enter_index + 1
            yield moves, Hall(self.slots[:i] + (one, ) + self.slots[i+1:])

    def exit(self, exit_index, one):
        for i in range(exit_index-1,-1,-1):
            if i in Hall.closed_slots or self.slots[i] is None: 
                continue
            
            else:
                if self.slots[i] == one:
                    moves = exit_index - i + 1
                    hall = Hall(self.slots[:i] +(None,) + self.slots[i+1:])
                    yield moves, Hall(self.slots[:i] +(None,) + self.slots[i+1:])
                break            

        
        for i in range(exit_index+1,len(self.slots)):
            if i in Hall.closed_slots or self.slots[i] is None: 
                continue
            else:
                if self.slots[i] == one:
                    moves = i - exit_index + 1
                    yield moves, Hall(self.slots[:i] +(None,) + self.slots[i+1:])
                break  

        
class Room(namedtuple("Room", ['owner', 'members'])):        
    @property
    def can_exit(self):
        return any(one != self.owner for one in self.members)

    @property
    def can_enter(self):
        return all([
            len(self.members) < ROOM_DEPTH,
            all(one==self.owner for one in self.members)
        ])
    
    def pop(self):
        *rest, member = self.members
        return member, Room(owner=self.owner, members=tuple(rest))
    
    def push(self, member):
        return Room(owner=self.owner, members=self.members + (member,))
                
        
class Game(namedtuple("State", ['rooms','hall'])):
    costs = {
        'A': 1,
        'B': 10,
        'C': 100,
        'D': 1000
    }
    
    def replace_room(self, room, index):
        rooms = tuple(room if i==index else r for i, r in enumerate(self.rooms))
        return Game(rooms=rooms, hall=self.hall)
        
    def enter_hallway(self, room_index):
        member, room = self.rooms[room_index].pop()
        
        new_rooms = tuple(room if i==room_index else r 
                          for i, r in enumerate(self.rooms))
        
        hall_index = (1 + room_index) * 2
        exit_cost = ROOM_DEPTH - len(room.members) - 1 
        
        for cost, new_hall in self.hall.enter(hall_index, member):
            yield Game.costs[member] * (cost + exit_cost), Game(rooms=new_rooms, hall=new_hall)
    
    def leave_hallway(self, room_index, member):
        hall_index = (1 + room_index) * 2
        
        new_rooms = tuple(r.push(member) if i==room_index else r 
                          for i, r in enumerate(self.rooms))
        
        enter_cost =  ROOM_DEPTH - len(self.rooms[room_index].members) - 1
        for cost, new_hall in self.hall.exit(hall_index, member):
            yield Game.costs[member] * (cost + enter_cost), Game(rooms=new_rooms, hall=new_hall)


    def __repr__(self):
        rooms = reversed(list(zip_longest(*[m.members for m in self.rooms], fillvalue='.')))
        s = ["#" * 13,
            '#' + ''.join('.' if l is None else l for l in self.hall.slots) + '#',
            '###' + '#'.join(next(rooms)) + '###'
        ]
        s += ['   ' + '#'.join(r) + '#' for r in rooms]
        return '\n'.join(s)
    
    def next_states(self):
        if self == FINAL_STATE:
            yield 0, self
            return

        for room_index, room in enumerate(self.rooms):
            if room.can_exit:
                yield from self.enter_hallway(room_index)
            if room.can_enter:
                yield from self.leave_hallway(room_index, room.owner)

Final_Rooms = tuple(Room(l, (l, l, l, l)) for l in 'ABCD')
FINAL_STATE = Game(Final_Rooms, Hall())

FINAL_STATE               

#############
#...........#
###A#B#C#D###
   A#B#C#D#
   A#B#C#D#
   A#B#C#D#

In [5]:
from functools import lru_cache

Part_Two = Game(
    rooms = (
        Room(owner='A', members=('B','D','D','C')),
        Room(owner='B', members=('D','B','C','B')),
        Room(owner='C', members=('A','A','B','D')),
        Room(owner='D', members=('C','C','A','A'))
    ),
    hall = Hall()
)

Part_One = Game(
    rooms = (
        Room(owner='A', members=('B', 'C')),
        Room(owner='B', members=('D', 'B')),
        Room(owner='C', members=('A', 'D')),
        Room(owner='D', members=('C', 'A'))
    ),
    hall = Hall
)

@lru_cache(maxsize=None)
def min_cost(game):
    if game == FINAL_STATE:
        return 0, FINAL_STATE
    
    costs = []
    
    for cost, state in game.next_states():
        if cost is not None:
            next_c = min_cost(state)
            if next_c:
                next_cost, next_state = next_c
                costs.append((cost + next_cost, next_state))

    if costs:
        costs = min(costs, key=lambda c: c[0])
        return costs

print(Part_Two)

min_cost(Part_Two)


#############
#...........#
###C#B#D#A###
   D#C#B#A#
   D#B#A#C#
   B#D#A#C#


(48708,
 #############
 #...........#
 ###A#B#C#D###
    A#B#C#D#
    A#B#C#D#
    A#B#C#D#)

### Some Tests

In [497]:
room = Room(owner='A', members=('B','C'))
r, room2 = room.pop()
assert(room == room2.push(r))
s, room3 = room2.pop()
assert(room2 == room3.push(s))
assert(room == room3.push(s).push(r))

assert(room.can_exit)
assert(room2.can_exit)
assert(room.can_enter is False)
assert(room2.can_enter is False)
assert(room3.can_enter)

room4 = Room(owner='A', members=('A','A'))
assert(room4.can_enter is False)
r, room5 = room4.pop()
assert(room5.can_enter)